In [2]:
from langchain_community.document_loaders import PyPDFLoader
import os
import getpass
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import HuggingFaceEmbeddings

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

loader = PyPDFLoader("~/Dropbox/LLM-agriculture/PDFs/433111_Fukushima.pdf")
pages = loader.load_and_split()


In [3]:
pages

[Document(page_content='○イチゴの病害  \n土壌病害  \n（１） 本県での発生状況  \n本県イチゴ に発生する 土壌病害では、炭疽病と萎黄病が比較的多く問題と なっている。 疫病\nはまれにみられる程度で、 萎凋病は露地育苗が ほとんどなくなったため、近年は ほとんど見 られ\nない。 \n土壌病害は、土壌中の植物残渣などに生存する病原菌により引き起こされ、その病徴は地下\n部だけでなく地上部まで進展し、株全体が枯死するため、甚大な被害になる恐れがある。  \n炭疽病は 2001年～2011年にやや多く発生し、現在でも 散見され ている。 \n萎黄病は 2007年～2011年にやや発生が多かったが、現在は常発ほ場で 見かける程度とな\nっている。  \n（２） 形態及び生態、被害の様子  \n○萎黄病（病原菌： Fusarium oxysporum  ） \n根から侵入し、導管部を侵す。典型的な病徴は 、小葉の大きさが不ぞろいとな り黄化し、葉は\n光沢を失い萎凋 する。最終的には、わい化または枯死に至る 。また、クラウンを切断すると導管\nの一部に褐変が見られる。  \n【伝搬方法】  \n土壌伝染・・・厚膜胞子が土壌中に残って伝染源となり、イチゴの根から侵入し感染・発病\nする。厚膜胞子は数年以上生存できる。  \n株伝染（潜在感染）・・・潜在感染している親株を使用すると、ランナーを経由して子苗に伝\n染する。  \n菌はイチゴのみに寄生し、キュウリ、トマト、ナス等には寄生しない。  \n【発病時期】  \n地温18～25℃で発生し、高温ほど病勢が進展する。菌の生育適温は 28℃。 \n \n \n \n \n \n \n \n \n写真1 萎黄病被害株       写真2 被害株に生じる奇形葉      写真3 被害株のクラウン  \n横断面', metadata={'source': '/Users/kaitokikuchi/Dropbox/LLM-agriculture/PDFs/433111_Fukushima.pdf', 'page': 0}),
 Document(page_content='○炭疽病（病原菌：  Glomerella cingulata  （Colletotrichum gloeosporioides 

In [6]:


def make_embedder(model_type, save_path):
    if model_type == "sup-simcse-ja-base":
        model_name = "cl-nagoya/sup-simcse-ja-base"
        model_kwargs = {'device': 'cpu'}
        encode_kwargs = {'normalize_embeddings': False}
        embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
            )
    else:  # Assuming OpenAIEmbeddings
        embedding_model = OpenAIEmbeddings()
    
    fs = LocalFileStore(save_path)
    cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
    return cached_embedder

def save_embeddings_and_ids_with_faiss(all_documents, cached_embedder, save_path):
    db = FAISS.from_documents(all_documents, cached_embedder)
    db.save_local(save_path)

model_type = "sup-simcse-ja-base"  # "sup-simcse-ja-base" or "OpenAIEmbeddings"
cached_embedder_dir = "data/cached_embeddings"
save_path = os.path.join(cached_embedder_dir, model_type)

cached_embedder = make_embedder(model_type, save_path)
save_embeddings_and_ids_with_faiss(pages, cached_embedder, save_path)

In [7]:
# クエリを受け取るたびにロードしなおさなくてよいようにmain実行時にすべてのデータをロードしておく
model_name = "cl-nagoya/sup-simcse-ja-base"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
            )
fs = LocalFileStore(save_path)
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_model, fs)
db = FAISS.load_local(save_path, cached_embedder)

In [14]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKeyIfNotSet')
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)


qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

In [17]:
query = "うどんこ病の防除策は何ですか？"
qa.invoke(query)

{'query': 'うどんこ病の防除策は何ですか？',
 'result': ' うどんこ病の防除策は、茎葉の過繁茂を防ぐために下葉や枯れ葉の摘葉を行うこと、適正な肥培管理を行うこと、湿度管理を適正に行うこと、そして罹病果実や罹病葉を見つけ次第除去することです。また、土壌病害の簡易診断法を用いて炭疽病や萎黄病を早期に発見し、適切な対策を行うことも重要です。さらに、土壌消毒や太陽熱土壌消毒法などの防除方法も有効です。'}